In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *


In [3]:
PATH="data/plantseedling/"
size=96
architecture=resnext101_64
batch_size=32

In [4]:
classes = ("Black-grass",
"Charlock",
"Cleavers",
"Common Chickweed",
"Common wheat",
"Fat Hen",
"Loose Silky-bent",
"Maize",
"Scentless Mayweed",
"Shepherds Purse",
"Small-flowered Cranesbill",
"Sugar beet")

In [5]:
??ImageClassifierData.from_paths

In [6]:
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

In [7]:
??ImageClassifierData.from_paths

In [8]:
def get_data(sz,bs):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH, trn_name="train",val_name="train",test_name="test", tfms=tfms, bs=bs)

In [ ]:
data = get_data(96,32)

In [ ]:
file_name = PATH+data.trn_ds.fnames[1]
file_name

In [ ]:
img = PIL.Image.open(file_name)
img

In [ ]:
img.size

In [ ]:
len(data.classes), data.classes


In [ ]:
len(data.trn_ds)


In [ ]:
len(data.test_ds)

In [ ]:
learn = ConvLearner.pretrained(architecture, data, precompute=True)

In [ ]:
lrf=learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
learn.fit(1e-2, 5)

In [ ]:
from sklearn import metrics

In [ ]:
learn.precompute=False

In [ ]:
learn.fit(1e-2, 5, cycle_len=1)

In [ ]:
learn.save('224_pre')

In [ ]:
learn.load('224_pre')

In [ ]:
learn.set_data(get_data(96, 32))
learn.freeze()

In [ ]:
%time learn.fit([1e-5, 1e-4,1e-2], 1, cycle_len=1)

In [ ]:
log_preds, y = learn.TTA()

In [ ]:
probs = np.mean(np.exp(log_preds), 0)
metrics.log_loss(y, probs)

In [ ]:
accuracy_np(probs, y)

In [ ]:
submission_csv = f'{PATH}sample_submission.csv'
submission_df = pd.read_csv(submission_csv)
submission_df.head()

In [ ]:
data.classes

In [ ]:
data.test_ds.fnames[:10]

In [ ]:
log_preds, y = learn.TTA(is_test=True)
probs = np.mean(np.exp(log_preds), 0)

In [ ]:
probs.shape 

In [ ]:
log_preds = learn.predict(is_test=True)

In [ ]:
log_preds.shape

In [ ]:
df = pd.DataFrame(log_preds)

In [ ]:
df.columns = data.classes

In [ ]:
df.columns[:5]

In [ ]:
df.insert(0, 'file', [o[5:] for o in data.test_ds.fnames])

In [ ]:
df[:100]

In [ ]:
df.to_csv(f'{PATH}submission-edit.csv')

In [ ]:
learn

In [ ]:
torch.cuda.is_available()

In [9]:
sz = 224; bs = 64

In [10]:
tfms = tfms_from_model(resnet50, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms= tfms, bs=bs, trn_name="train",val_name="train",test_name="test")
learn = ConvLearner.pretrained(resnet50, data )
% time learn.fit( 1e-2, 10, cycle_len=1)

# deeper model like resnet 50 

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                 
    0      1.238136   0.720238   0.778095  
    1      0.874334   0.48716    0.846339                  
    2      0.698653   0.39765    0.875923                  
    3      0.606405   0.351258   0.885714                  
    4      0.556155   0.313827   0.896964                  
    5      0.513424   0.281897   0.908006                  
    6      0.473598   0.261061   0.917798                  
    7      0.446264   0.238311   0.924464                  
    8      0.425674   0.222596   0.925923                  
    9      0.426182   0.209881   0.931339                  

CPU times: user 25min 4s, sys: 2min 36s, total: 27min 41s
Wall time: 7min 22s


[0.20988137, 0.9313392853736877]

In [11]:
%time learn.fit(1e-2, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                  
    0      0.378422   0.199802   0.938839  
    1      0.390705   0.189751   0.941548                  
    2      0.379816   0.18567    0.940923                  

CPU times: user 7min 31s, sys: 45.8 s, total: 8min 17s
Wall time: 2min 12s


[0.18566954, 0.9409226187070211]

In [12]:
%time learn.fit(1e-2, 2, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                  
    0      0.362477   0.180757   0.943423  
    1      0.342626   0.171911   0.946131                  

CPU times: user 4min 59s, sys: 31.4 s, total: 5min 31s
Wall time: 1min 28s


[0.17191081, 0.9461309520403544]

In [13]:
%time learn.fit(1e-2, 6, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                  
    0      0.344687   0.159511   0.950298  
    1      0.332629   0.154264   0.953006                  
    2      0.332015   0.150392   0.954048                  
    3      0.322758   0.146094   0.952381                  
    4      0.324819   0.14285    0.957798                  
    5      0.317908   0.140759   0.953631                  

CPU times: user 15min 4s, sys: 1min 35s, total: 16min 40s
Wall time: 4min 28s


[0.14075923, 0.9536309520403544]

In [14]:
learn.unfreeze()
learn.bn_freeze(True)

In [15]:
%time learn.fit([1e-5, 1e-4,1e-2], 2, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                  
    0      0.240767   0.104698   0.966339  
    1      0.195572   0.090628   0.968839                  

CPU times: user 5min 44s, sys: 48.9 s, total: 6min 33s
Wall time: 2min 27s


[0.09062847, 0.9688392853736878]

In [16]:
%time learn.fit([1e-5, 1e-4,1e-2], 2, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                  
    0      0.167533   0.078359   0.97625   
    1      0.160409   0.073788   0.976131                  

CPU times: user 5min 43s, sys: 49.2 s, total: 6min 32s
Wall time: 2min 27s


[0.07378846, 0.9761309520403544]

In [17]:
%time learn.fit([1e-5, 1e-4,1e-2], 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                  
    0      0.135441   0.066053   0.978542  

CPU times: user 2min 50s, sys: 24.3 s, total: 3min 14s
Wall time: 1min 13s


[0.0660532, 0.9785416666666666]

In [26]:
%time learn.fit([1e-5, 1e-4,1e-2], 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      0.126963   0.060394   0.982917  

CPU times: user 2min 57s, sys: 24.7 s, total: 3min 22s
Wall time: 1min 13s


[0.06039395, 0.9829166666666667]

In [27]:
%time learn.fit([1e-5, 1e-4,1e-2], 6, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                  
    0      0.131866   0.056053   0.981667  
    1      0.10847    0.048641   0.984583                  
    2      0.103963   0.046105   0.987083                  
    3      0.102857   0.044553   0.985625                  
    4      0.096433   0.038187   0.987292                   
    5      0.085206   0.035204   0.990417                   

CPU times: user 20min 56s, sys: 2min 34s, total: 23min 31s
Wall time: 7min 43s


[0.035204288, 0.9904166666666666]

In [28]:
%time learn.fit([1e-5, 1e-4,1e-2], 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      0.082088   0.032235   0.992292  
    1      0.088611   0.035806   0.9875                     
    2      0.076923   0.027249   0.991875                   
    3      0.083924   0.03356    0.989583                   
    4      0.069519   0.023913   0.993542                   
    5      0.05778    0.019832   0.994167                   
    6      0.048488   0.019012   0.994792                   

CPU times: user 24min 55s, sys: 3min 3s, total: 27min 58s
Wall time: 9min 2s


[0.019012036, 0.9947916666666666]

In [29]:
%time learn.fit([1e-5, 1e-4,1e-2], 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      0.061211   0.021187   0.993333  
    1      0.059727   0.020845   0.993542                   
    2      0.046555   0.016484   0.995                      
    3      0.062108   0.019779   0.993542                   
    4      0.0547     0.021614   0.992917                   
    5      0.039795   0.014501   0.996667                   
    6      0.043208   0.013273   0.996458                   

CPU times: user 25min 38s, sys: 3min 2s, total: 28min 40s
Wall time: 9min 6s


[0.013272746, 0.9964583333333333]

In [30]:
learn.save('train_plant')

In [31]:
log_preds, y = learn.TTA(is_test=True)
probs = np.mean(np.exp(log_preds), 0)

In [32]:
log_preds = learn.predict(is_test=True)

In [33]:
df = pd.DataFrame(log_preds)

In [34]:
df.columns = data.classes

In [35]:
df.insert(0, 'file', [o[5:] for o in data.test_ds.fnames])

In [36]:
df.head()

,file,Black-grass,Charlock,Cleavers,Common Chickweed,Common wheat,Fat Hen,Loose Silky-bent,Maize,Scentless Mayweed,Shepherds Purse,Small-flowered Cranesbill,Sugar beet
0,79e5ea8fa.png,-14.130864,-15.659530,-13.169687,-19.136372,-0.000044,-10.433470,-14.520191,-12.891104,-16.645193,-21.151936,-19.147923,-11.650206
1,3a909ead8.png,-23.337969,-18.067305,-18.355675,-20.662127,-23.234419,-23.603935,-24.046673,-21.394272,-22.687586,-13.063572,-0.000002,-29.303175
2,599691cd9.png,-20.228548,-14.229105,-16.481333,-12.562889,-21.638103,-19.577677,-18.705122,-18.202511,-16.333393,-7.062665,-0.000861,-24.172028
3,5ca2687a4.png,-21.987843,-10.709475,-19.678297,-17.269295,-15.573669,-16.792709,-28.336367,-0.000023,-22.035391,-16.825544,-23.086893,-13.638818
4,3dd52bd2a.png,-22.048820,-16.709677,-17.898098,-13.566433,-23.976486,-20.499456,-19.357128,-19.316713,-18.230591,-8.011662,-0.000334,-26.211948


In [37]:
df.to_csv(f'{PATH}submission-edit-newtrain.csv')